In [ ]:
gpu=0
re_calib=False
re_bias_y=False
dataset='ackley'
log_root='int_log'
model='small'

In [ ]:
# Imports

from argparse import Namespace
import copy
import pickle
from pathlib import Path

import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

from ackley import Ackley
from probo import NelderMeadAcqOptimizer, SimpleBo
from probo.util.misc_util import dict_to_namespace
from branin import branin, get_branin_domain_nd
from penn_sklearn import SklearnPenn

import sys
sys.path.append('../')
from models import *
from utils import *

In [ ]:
class Args:
    def __init__(self):
        self.log_root=log_root
        self.incremental = True # If true we don't retrain the model every step
        self.dataset = dataset  # This is branin or ackley
        
        self.re_calib = re_calib
        self.re_bias_f = False
        self.re_bias_y = re_bias_y

        # Modeling parameters
        self.model = model
        self.learning_rate = 1e-3
        self.num_bins = 0
        self.knn = 10

        # Run related parameters
        self.gpu = gpu
        self.num_iter = 1000
        self.run_label = 0
        self.num_run = 10
        self.flow_skip = 1
        
args = Args()

device = torch.device('cuda:%d' % args.gpu)
args.device = device

In [ ]:


start_time = time.time()

if args.num_bins == 0:
    eval_bias = eval_bias_knn
    assert args.knn >= 10 and args.knn % 2 == 0
    

while True:
    args.name = '%s/model=%s-%r-%r-%r-%r-bin=%d-%d-run=%d' % \
        (args.dataset, args.model, args.incremental, args.re_calib, args.re_bias_f, args.re_bias_y, args.num_bins, args.knn, args.run_label)
    args.log_dir = os.path.join(args.log_root, args.name)
    if not os.path.isdir(args.log_dir):
        os.makedirs(args.log_dir)
        break
    args.run_label += 1
print("Run number = %d" % args.run_label)
writer = SummaryWriter(args.log_dir)
log_writer = open(os.path.join(args.log_dir, 'results.txt'), 'w')

global_iteration = 0
random.seed(args.run_label)  # Set a different random seed for different run labels
torch.manual_seed(args.run_label)

def log_scalar(name, value, epoch):
    writer.add_scalar(name, value, epoch)
    log_writer.write('%f ' % value)

In [ ]:
# Define probabilistic ensemble of neural networks (PENN) model

class TorchPenn:
    """
    Probabilistic ensemble neural network (PENN) model implemented in
    scikit-learn.
    """

    def __init__(self, params=None, verbose=True):
        """
        Parameters
        ----------
        params : Namespace_or_dict
            Namespace or dict of parameters for this model.
        verbose : bool
            If True, print description string.
        """
        self.set_params(params)
        if verbose:
            print(f'*[INFO] Model=SklearnPenn with params={self.params}')
        self.ensemble = []
        self.optims = []
        
    def set_params(self, params):
        """Set self.params, the parameters for this model."""
        params = dict_to_namespace(params)

        self.params = Namespace()
        self.params.n_ensemble = getattr(params, 'n_ensemble', 5)
        self.params.hls = getattr(params, 'hls', (20, 30, 40))
        self.params.max_iter = getattr(params, 'max_iter', 500)
        self.params.alpha = getattr(params, 'alpha', 0.01)
        self.params.trans_x = getattr(params, 'trans_x', False)

    def set_data(self, data):
        """Set self.data."""
        self.data = copy.deepcopy(data)

    @ignore_warnings(category=ConvergenceWarning)
    def inf(self, data):
        global global_iteration
        """Set data, run inference."""
        self.set_data(data)
        x_train = torch.from_numpy(np.array(self.data.x)).type(torch.float32).to(args.device)
        y_train = torch.from_numpy(np.array(self.data.y)).view(-1, 1).type(torch.float32).to(args.device)
        print(x_train.shape, y_train.shape)
        
        if not args.incremental or len(self.ensemble) == 0:
            self.ensemble = [model_list[args.model](x_train.shape[1]).to(device) for i in range(self.params.n_ensemble)]
            self.optims = [optim.Adam(self.ensemble[i].parameters(), lr=args.learning_rate) for i in range(self.params.n_ensemble)]
            if args.re_calib or args.re_bias_f or args.re_bias_y:
                for model in self.ensemble:
                    model.recalibrator = RecalibratorOnline(model, args, re_calib=args.re_calib, re_bias_f=args.re_bias_f, re_bias_y=args.re_bias_y)
        for model, optimizer in zip(self.ensemble, self.optims):
            # Define model and optimizer
            # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.num_epoch // 20, gamma=0.9) 
            # train_bb_iter = itertools.cycle(train_bb_loader)
        
            if args.re_bias_f or args.re_bias_y or args.re_calib:
                randperm = torch.randperm(x_train.shape[0])
                x_train, y_train = x_train[randperm], y_train[randperm]
                num_train = x_train.shape[0] * 2 // 3
                x_val, y_val = x_train[num_train:], y_train[num_train:]
                x_train, y_train = x_train[:num_train], y_train[:num_train]
            
            model.train()
            # print(args.num_epoch)
            for epoch in range(args.num_iter):
                # Minimize L2
                optimizer.zero_grad()
                loss_l2 = eval_l2(model, (x_train, y_train), args)
                loss_l2.mean().backward()
                optimizer.step()
                # scheduler.step()
                # writer.add_scalar('loss_l2', loss_l2.mean(), global_iteration)
                
                if model.recalibrator is not None:
                    model.recalibrator.train_step((x_val, y_val))
                global_iteration += 1
                # if epoch % 499 == 0:
                #    print("Time elapsed = %.3f, loss=%.3f" % (time.time() - start_time, loss_l2.mean().cpu().item())) 
            model.eval()
            


    def post(self, s):
        """Return one posterior sample"""
        return np.random.choice(self.ensemble)

    def gen_list(self, x_list, z, s, nsamp):
        """
        Draw nsamp samples from generative process, given list of inputs x_list,
        and posterior sample z.

        Parameters
        ----------
        x_list : list
            List of numpy ndarrays each with shape=(-1,).
        z : Namespace
            Namespace of GP hyperparameters.
        s : int
            [FOR THIS MODEL, YOU CAN IGNORE THIS.] The seed, a positive integer.
        nsamp : int
            The number of samples to draw from generative process.

        Returns
        -------
        list
            A list with len=len(x_list) of numpy ndarrays, each with shape=(nsamp,).
        """
        # print(len(x_list), nsamp)
        with torch.no_grad():
            pred_list = [
                z(torch.from_numpy(x.reshape(1, -1)).type(torch.float32).to(device).repeat(nsamp, 1)).cpu().numpy().reshape(-1)
                for x in x_list
            ]
        return pred_list


In [ ]:
# BO Setup

# define function

if args.dataset == 'branin':
    n_dim = 40
    f = lambda x: np.sum([branin(x[2 * i : 2 * i + 2]) for i in range(n_dim // 2)])

    # define model
    model = TorchPenn()

    # define acqfunction
    acqfunction = {'acq_str': 'ts', 'n_gen': 500}

    # define acqoptimizer
    domain = get_branin_domain_nd(n_dim)
    acqoptimizer = NelderMeadAcqOptimizer({'rand_every': 10, 'max_iter': 200, 'jitter': True}, domain)
else:
    assert args.dataset == 'ackley'
    
    # define function
    n_dim = 10
    f = Ackley(n_dim)

    # define model
    model = SklearnPenn()

    # define acqfunction
    acqfunction = {'acq_str': 'ts', 'n_gen': 500}

    # define acqoptimizer
    domain = f.get_domain()
    acqoptimizer = NelderMeadAcqOptimizer({'rand_every': 10, 'max_iter': 200, 'jitter': True}, domain)

# define initial dataset
n_init = 100
data = Namespace()
data.x = domain.unif_rand_sample(n_init)
data.y = [f(x) for x in data.x]

# define BO
n_iter = 200
bo = SimpleBo(f, model, acqfunction, acqoptimizer, data=data, params={'n_iter': n_iter}, seed=args.run_label)

In [ ]:
# Run BO

results = bo.run()

In [ ]:
# Save results

save_dir = Path(f'results/branin{n_dim}_sklp_{args.run_label}')
save_dir.mkdir(parents=True, exist_ok=True)

with open(save_dir / "results.pkl", "wb") as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
ys = vars(vars(results)['data'])['y']

In [ ]:
for y in ys:
    log_writer.write('%f ' % y)
log_writer.flush()

In [ ]:
min_y = [ys[0]]
for y in ys[1:]:
    if y < min_y[-1]:
        min_y.append(y)
    else:
        min_y.append(min_y[-1])

In [ ]:
plt.plot(min_y)
plt.show()